# The Database: `pgstac`

`pgstac` is a PostgreSQL extension that enables STAC metadata management in a PostgreSQL database.
eoAPI is useful to many organizations because the other components are configured to work seamlessly with STAC metadata that is stored in your `pgstac` database.

Your STAC metadata generation and ingestion workflow will probably not take place in a notebook but the basic steps will be the same:
1. Generate STAC collection record
2. Add new record it to the collections table with `Loader.load_collections`
3. Generate STAC item records
4. Add new records to the items table with `Loader.load_items`

In this notebook you will learn how to interact with a `pgstac` database:
- Create and upload a collection
- Upload items
- Delete an item

Fill in the input boxes below to get started with your own personal Sentinel-2 STAC collection!

In [3]:
import ipywidgets as widgets
from IPython.display import display

username_input = widgets.Text(
    value="", placeholder="Enter your username", description="username:", disabled=False
)

lat_input = widgets.BoundedFloatText(
    value=38.9,
    min=-90,
    max=90,
    placeholder="enter the latitude of your hometown",
    description="latitude:",
    disabled=False,
)
lon_input = widgets.BoundedFloatText(
    value=-77.0,
    min=-180,
    max=180,
    placeholder="enter the longitude of your hometown",
    description="longitude:",
    disabled=False,
)
db_host_input = widgets.Text(
    value="",
    placeholder="Enter database host address",
    description="db host:",
    disabled=False,
)
db_user_input = widgets.Text(
    value="pgstac_user",
    placeholder="Enter database username",
    description="db user:",
    disabled=False,
)
db_password_input = widgets.Text(
    value="",
    placeholder="Enter database password",
    description="db password:",
    disabled=False,
)

# Display the widgets
display(username_input)
display(lat_input)
display(lon_input)
display(db_host_input)
display(db_user_input)
display(db_password_input)

Text(value='', description='username:', placeholder='Enter your username')

BoundedFloatText(value=38.9, description='latitude:', max=90.0, min=-90.0)

BoundedFloatText(value=-77.0, description='longitude:', max=180.0, min=-180.0)

Text(value='', description='db host:', placeholder='Enter database host address')

Text(value='pgstac_user', description='db user:', placeholder='Enter database username')

Text(value='', description='db password:', placeholder='Enter database password')

In [4]:
from datetime import datetime

import pystac_client
from pystac import Collection, Extent, SpatialExtent, TemporalExtent
from pypgstac.db import PgstacDB
from pypgstac.load import Loader, Methods
from shapely.geometry import Point

# deployed stac-fastapi-pgstac API
stac_api_endpoint = "https://pj44p72a3g.execute-api.us-west-2.amazonaws.com"

## Load a collection object

Start by creating a collection that will contain items within ~2 degrees of your provided location between January 1 2025 and April 18 2025.

In [5]:
collection_id = f"{username_input.value}-sentinel-2-c1-l2a"
bbox = Point(lon_input.value, lat_input.value).buffer(2).bounds
temporal_extent = [datetime(2025, 1, 1), datetime(2025, 4, 18)]

my_collection = Collection(
    id=collection_id,
    description="my collection",
    extent=Extent(
        spatial=SpatialExtent([[*bbox]]),
        temporal=TemporalExtent([temporal_extent]),
    ),
)
my_collection

<Collection id=test-sentinel-2-c1-l2a>

To connect to the `pgstac` database and upload data we need to generate the `dsn` and instantiate a `PgstacDB` and `Loader` pair.

In [30]:
db_creds = {
    "password": db_password_input.value,
    "dbname": "pgstac",
    "engine": "postgres",
    "port": 5432,
    "host": db_host_input.value,
    "username": db_user_input.value,
}
pgstac_dsn = "postgresql://{username}:{password}@{host}:{port}/{dbname}".format(
    **db_creds
)

db = PgstacDB(dsn=pgstac_dsn)
loader = Loader(db)

The `load_collections` method accepts an iterable of STAC collection dictionaries or a file path to a collection.json file or a .ndjson file with multiple collection records. You already have your `pystac.Collection` in memory in this session so you can just write it to a dictionary and upload it (in a list). 

The `upsert` method will add your collection to the `collections` table if it does not exist and update any fields with new values if the record does already exist.

In [13]:
loader.load_collections([my_collection.to_dict()], insert_mode=Methods.upsert)

Now if you run a query on the database you will find a record for your collection!

In [19]:
list(db.query(f"SELECT id from collections where id = '{my_collection.id}';"))

[('hrodmn-sentinel-2-c1-l2a',)]

## Upload items

You already learned how to create STAC items from scratch in the [STAC metadata](./stac_metadata.ipynb) notebook so you get to take a shortcut for this one. Instead of creating STAC metadata from scratch you can just copy some from an existing STAC collection - this will be sufficient for our exercises during the workshop.

This code will find Sentinel-2 L2A items for the first few months of 2025 that intersect the bounding box of a 2 degree radius around the coordinate that you entered at the top of the notebook.

In [20]:
source_client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")

search = source_client.search(
    collections="sentinel-2-c1-l2a",
    bbox=bbox,
    datetime=temporal_extent,
    limit=100,  # pagination limit
)

items = search.item_collection()

print(len(items))

1430


You are going to upload these items to the items table in the `pgstac` database but to do so you need to make sure the items' collection ID matches an existing collection - set it to match the collection that you uploaded in the previous step.

In [ ]:
# override the collection id to match your new collection
for item in items:
    item.set_collection(my_collection)

items[0]

Now you can use the `load_items` method to upload the list of STAC item dictionaries to the items table in the `pgstac` database. The `insert_ignore` method will upload any items that do not exist in the table and skip records that already exist.

In [22]:
loader.load_items([item.to_dict() for item in items], insert_mode=Methods.insert_ignore)

Hooray you uploaded some items! Now run a query to verify that the records landed in the items table.

In [27]:
n_items = db.query_one(
    f"SELECT COUNT(*) FROM items where collection = '{my_collection.id}';"
)
print(f"there are {n_items} items")

there are 1430 items


Here is your first glimpse of the power of STAC metadata. You just uploaded these items to the database and now you can browse the data in a beautiful interface without doing any more work!

[Radiant Earth](https://radiant.earth/) built a tool called [STAC Browser](https://radiantearth.github.io/stac-browser/#/?.language=en) that is a human-readable interface to a STAC API. Your collection is immediately availble in the STAC API so you can browse it in STAC Browser. Take a tour around the STAC Browser page for your collection. 

In [6]:
from IPython.display import IFrame

IFrame(
    f"https://radiantearth.github.io/stac-browser/#/external/{stac_api_endpoint}/collections/{my_collection.id}",
    1500,
    1000,
)

## Delete an item

It is less common but you might want to delete a STAC record some day. There are not yet any `pypgstac` functions for doing this but it is relatively easy to do via PostgreSQL query.

Try deleting the last item in the list.

In [28]:
with db.connect() as conn:
    cur = conn.cursor()
    cur.execute(f"DELETE FROM items where id = '{items[-1].id}';")
    cur.close()
    conn.commit()

In [32]:
new_n_items = db.query_one(
    f"SELECT COUNT(*) FROM items where collection = '{my_collection.id}';"
)
print(f"now there are {new_n_items} items")

there are now 1429 items
